In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
def print_metrics(y_test,predictions,model):
    print('\033[1m' + model + '\033[0m' + '\n\n')
    print('\033[1m' + 'Classification Report - ' + '\033[0m')
    print(classification_report(y_test,predictions))
    print('\033[1m' + 'ROC AUC Score - ' + '\033[0m')
    print(roc_auc_score(y_test,predictions))
    print('\033[1m' + 'Accuracy Score - ' + '\033[0m')
    print(accuracy_score(y_test,predictions))
    
    

def dtree(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    dt = DecisionTreeClassifier()
    dt.fit(X_train,y_train)
    predictions_dt = dt.predict(X_test)
    print_metrics(y_test,predictions_dt,'Decision Tree')
    
def xboost(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    boost = XGBClassifier(eval_metric='auc')
    boost.fit(X_train,y_train)
    predictions_boost = boost.predict(X_test)
    print_metrics(y_test,predictions_boost,'XGBoost')
    
def rforrest(X,y,n):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    rfc = RandomForestClassifier(n_estimators=n,n_jobs=-1)
    rfc.fit(X_train,y_train)
    predictions_rfc = rfc.predict(X_test)
    print_metrics(y_test,predictions_rfc,'Random Forrest')
    
def nbayes(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    nb = GaussianNB()
    nb.fit(X_train,y_train)
    predictions_nb = nb.predict(X_test)
    print_metrics(y_test,predictions_nb,'Naive Bayes')

In [8]:
X = pd.read_csv('../input/final_data.csv')
Y = pd.read_csv('../input/final_target.csv', header=None)

In [12]:
Y = Y[1]

In [ ]:
xboost(X,Y)